In [2]:
import os, re

import numpy as np
import pandas as pd
import json

import tsfresh
from tsfresh import extract_features
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor

np.random.seed(123)

labels = [5, 6, 7]
basedir = 'C:/Users/tmhnguyen/Documents/lalamove/lalamove/data/Clean_extracted_240221_cal_combined/train'
with open(basedir + '/../data_split_params.json', 'r') as file:
    features = json.load(file)['FEATURES']
print(features)
WINDOW = 5 # seconds
SAMPLING_RATE = 20 # hz
window_length = WINDOW * SAMPLING_RATE

{'5': ['z_gyro_clean', 'y_gyro_clean', 'x_gyro_clean', 'x_acc_clean', 'y_acc_clean', 'z_acc_clean', 'acceleration', 'speed_kmh'], '6': ['z_gyro_clean', 'y_gyro_clean', 'x_gyro_clean', 'x_acc_clean', 'y_acc_clean', 'z_acc_clean', 'acceleration', 'speed_kmh'], '7': ['z_gyro_clean', 'y_gyro_clean', 'x_gyro_clean', 'x_acc_clean', 'y_acc_clean', 'z_acc_clean', 'acceleration', 'speed_kmh']}


In [2]:
# # extracting features from nothing
# for label_title in labels:
#     # load our data shape (sample_id, timestamp, time series)
#     label_ = []
#     data_ = []
#     data_files = [f for f in os.listdir(basedir + f'/{label_title}/') if '.npy' in f]
#     print('data files', data_files)

#     for data_file in data_files:
#         # data_file = basedir + f'/{label_title}/chunk_1.npy'
#         chunk_id = data_file.split('.')[0].split('_')[1]
#         print('chunk id', chunk_id)

#         data = np.load(basedir + f'/{label_title}/' + data_file)
#         print('x', data.shape)

#         label_file = basedir + f'/{label_title}/train_label_{label_title}_chunk_{chunk_id}.csv'
#         label = pd.read_csv(label_file)

#         pos = label[(label.label == 1) & (label.type == 0)]
#         # pos_1 = label[(label.label == 1) & (label.type == 1)]
#         pos_1 = label[(label.label == 1) & (label.type == 1)].sample(n=len(pos)*5, replace=False, random_state=123)
#         neg = label[label.label == 0].sample(n=len(pos) + len(pos_1), replace=False, random_state=123)
#         label_tmp = pd.concat((pos, neg))
#         data_tmp = data[label_tmp.index] # previously made a mistake here, which wrongly mapped the features to wrong labels
#         label_tmp = label_tmp.reset_index() # should reset index after mapping, NOT before

#         label_.append(label_tmp)
#         data_.append(data_tmp)

#     label_ = pd.concat(label_).reset_index().label
#     data_ = np.concatenate(data_)
#     print('y', label_.shape, data_.shape)

#     # Create a list of column names for the DataFrame
#     column_names = ['id', 'timestamp'] + features[str(label_title)]

#     # Reshape the numpy array to have 2 dimensions
#     reshaped_array = data_.reshape(-1, data_.shape[2])

#     # Create a pandas DataFrame
#     my_df = pd.DataFrame(reshaped_array, columns=column_names[2:])
#     print(my_df.shape)

#     # Assign sample ids and timestamps to the DataFrame
#     num_samples = data_.shape[0]
#     num_timestamps = data_.shape[1]
#     my_df['id'] = np.repeat(np.arange(num_samples), num_timestamps)
#     my_df['timestamp'] = np.tile(np.arange(num_timestamps), num_samples)
#     # my_df = my_df[column_names]

#     # Print the resulting DataFrame
#     Distributor = MultiprocessingDistributor(n_workers=4,
#                                             disable_progressbar=False,
#                                             progressbar_title="Feature Extraction")
    
              
#     print('Extracting feature only...', len(my_df))
#     # extract all possible features
#     temp = extract_features(my_df, 
#                             column_id='id', 
#                             column_sort='timestamp', 
#                             column_kind=None, 
#                             column_value=None, 
#                             impute_function=impute)    
    
#     # calculate independent p-value for relevance
#     relevance_table = calculate_relevance_table(temp, label_, fdr_level=0.5)

#     # filter relevant features (True if the Benjamini Hochberg procedure rejected the null hypothesis [the feature is not relevant])
#     relevance_table = relevance_table[relevance_table['relevant'] == True]

#     # Among relevant features, sort out top window_length0 with smallest p-values
#     sorted_relevance_table = relevance_table.sort_values(by='p_value', ascending=True) 
#     top_extracted_features = sorted_relevance_table.head(500)
#     temp = temp[top_extracted_features.feature]

#     # save the features setting
#     kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(temp)
#     with open(basedir + f'/{label_title}/feature_config_{label_title}.json', 'w') as file:
#         json.dump(kind_to_fc_parameters, file, indent=4)
#     print(kind_to_fc_parameters)
    
#     ###########################################
#     # extract features using the saved features
#     for data_file in data_files:
#         # data_file = basedir + f'/{label_title}/chunk_1.npy'
#         chunk_id = data_file.split('.')[0].split('_')[1]
#         print('chunk id', chunk_id)

#         data = np.load(basedir + f'/{label_title}/' + data_file)
#         print('x', data.shape)

#         label_file = basedir + f'/{label_title}/train_label_{label_title}_chunk_{chunk_id}.csv'
#         label = pd.read_csv(label_file)
#         print('y', label.shape)

#         # Create a list of column names for the DataFrame
#         column_names = ['id', 'timestamp'] + features[str(label_title)]

#         # Reshape the numpy array to have 2 dimensions
#         reshaped_array = data.reshape(-1, data.shape[2])

#         # Create a pandas DataFrame
#         my_df = pd.DataFrame(reshaped_array, columns=column_names[2:])
#         print(my_df.shape)

#         # Assign sample ids and timestamps to the DataFrame
#         num_samples = data.shape[0]
#         num_timestamps = data.shape[1]
#         my_df['id'] = np.repeat(np.arange(num_samples), num_timestamps)
#         my_df['timestamp'] = np.tile(np.arange(num_timestamps), num_samples)
#         # my_df = my_df[column_names]

#         # Print the resulting DataFrame
#         print(my_df.shape)
#         temp = extract_features(my_df, 
#                                 kind_to_fc_parameters=kind_to_fc_parameters, 
#                                 column_id='id', 
#                                 column_sort='timestamp', 
#                                 column_kind=None, 
#                                 column_value=None, 
#                                 impute_function=impute)
#         temp.to_csv(basedir + f'/{label_title}/extract_features_{label_title}_{chunk_id}.csv')
#         print('extracted shape', temp.shape)

data files ['chunk_1.npy', 'chunk_2.npy', 'chunk_3.npy', 'chunk_4.npy']
chunk id 1
x (102400, 100, 8)
chunk id 2
x (102400, 100, 8)
chunk id 3
x (102400, 100, 8)
chunk id 4
x (90468, 100, 8)
y (15897,) (15897, 100, 8)
(1589700, 8)
Extracting feature only... 1589700


Feature Extraction: 100%|██████████| 60/60 [09:39<00:00,  9.66s/it]  


{'speed_kmh': {'agg_linear_trend': [{'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'mean'}, {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'min'}, {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'min'}, {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'max'}, {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'max'}, {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'mean'}, {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'max'}, {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'mean'}, {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'min'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'max'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'mean'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'min'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'mean'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'max'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'min'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'var'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'var'}, {'attr': 'intercept', 'chunk_len': 50,

Feature Extraction: 100%|██████████| 60/60 [11:23<00:00, 11.39s/it] 


extracted shape (102400, 500)
chunk id 2
x (102400, 100, 8)
y (102400, 6)
(10240000, 8)
(10240000, 10)


...

extracted shape (102400, 500)
chunk id 3
x (102400, 100, 8)
y (102400, 6)
(10240000, 8)
(10240000, 10)


Feature Extraction: 100%|██████████| 60/60 [11:18<00:00, 11.31s/it] 


extracted shape (102400, 500)
chunk id 4
x (91068, 100, 8)
y (91068, 6)
(9106800, 8)
(9106800, 10)


Feature Extraction: 100%|██████████| 60/60 [09:40<00:00,  9.67s/it] 


extracted shape (91068, 500)


In [27]:
start_chunk_ids = {5: 5,
                   6: 6,
                   7: 5}

# extracting features using a predined list of features
for label_title in labels:
    # load the features setting
    with open(basedir + f'/feature_config.json', 'r') as file:
        kind_to_fc_parameters = json.load(file)
    print(kind_to_fc_parameters)
    
    pattern = r'chunk_([5-9]|[1-9]\d+)\.npy'  # Regex pattern to match the file names >= 5
    data_files = [file for file in os.listdir(basedir + f'/{label_title}/') if re.match(pattern, file)]
    for data_file in data_files:
        # data_file = basedir + f'/{label_title}/chunk_1.npy'
        chunk_id = data_file.split('.')[0].split('_')[1]
        print('chunk id', chunk_id)

        data = np.load(basedir + f'/{label_title}/' + data_file)
        print('x', data.shape)

        label_file = basedir + f'/{label_title}/train_label_{label_title}_chunk_{chunk_id}.csv'
        label = pd.read_csv(label_file)
        print('y', label.shape)

        # Create a list of column names for the DataFrame
        column_names = ['id', 'timestamp'] + features[str(label_title)]

        # Reshape the numpy array to have 2 dimensions
        reshaped_array = data.reshape(-1, data.shape[2])

        # Create a pandas DataFrame
        my_df = pd.DataFrame(reshaped_array, columns=column_names[2:])
        print(my_df.shape)

        # Assign sample ids and timestamps to the DataFrame
        num_samples = data.shape[0]
        num_timestamps = data.shape[1]
        my_df['id'] = np.repeat(np.arange(num_samples), num_timestamps)
        my_df['timestamp'] = np.tile(np.arange(num_timestamps), num_samples)
        # my_df = my_df[column_names]

        # Print the resulting DataFrame
        print(my_df.shape)
        temp = extract_features(my_df, 
                                kind_to_fc_parameters=kind_to_fc_parameters, 
                                column_id='id', 
                                column_sort='timestamp', 
                                column_kind=None, 
                                column_value=None, 
                                impute_function=impute)
        temp.to_csv(basedir + f'/{label_title}/extract_features_{label_title}_{chunk_id}.csv')
        print('extracted shape', temp.shape)

{'y_acc_clean': {'linear_trend': [{'attr': 'stderr'}], 'agg_linear_trend': [{'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'max'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'min'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'mean'}, {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'var'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'max'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'mean'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'min'}, {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'var'}, {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'var'}, {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'var'}, {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'var'}], 'variance': None, 'standard_deviation': None, 'change_quantiles': [{'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.6}, {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.6}, {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.4}, {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2}, {'f_agg': 'mean', 'isabs': True, 'qh': 0.6,

Feature Extraction: 100%|██████████| 60/60 [12:07<00:00, 12.13s/it] 


extracted shape (102400, 500)
chunk id 2
x (102400, 100, 8)
y (102400, 6)
(10240000, 8)
(10240000, 10)


Feature Extraction: 100%|██████████| 60/60 [11:48<00:00, 11.80s/it]  


extracted shape (102400, 500)
chunk id 3
x (102400, 100, 8)
y (102400, 6)
(10240000, 8)
(10240000, 10)


...

extracted shape (102400, 500)
chunk id 3
x (102400, 100, 8)
y (102400, 6)
(10240000, 8)
(10240000, 10)


Feature Extraction: 100%|██████████| 60/60 [09:45<00:00,  9.76s/it] 


extracted shape (102400, 500)
chunk id 4
x (91068, 100, 8)
y (91068, 6)
(9106800, 8)
(9106800, 10)


Feature Extraction: 100%|██████████| 60/60 [09:03<00:00,  9.06s/it] 


extracted shape (91068, 500)


In [ ]:
# ## MERGING FEATURE SET

# import json 


# # compare the similarity of three sets of features
# labels = [5, 6, 7]
# feature_sets = []
# basedir = "C:/Users/tmhnguyen/Documents/lalamove/lalamove/data/Clean_extracted_240221_cal/train"

# for label in labels:
#     with open(basedir + f'/{label}/feature_config_{label}.json', 'r') as file:
#         kind_to_fc_parameters = json.load(file)
#         feature_sets.append(kind_to_fc_parameters)
# def merge_dicts(list_of_dicts):
#     dict1, dict2, dict3 = list_of_dicts
#     result = {}
#     all_keys = set(list(dict1.keys()) + list(dict2.keys()) + list(dict3.keys()))
#     unique_branches = 0

#     for key in all_keys:
#         if key in dict1 and isinstance(dict1[key], dict) and key in dict2 and isinstance(dict2[key], dict) and key in dict3 and isinstance(dict3[key], dict):
#             merged_value, sub_branches = merge_dicts([dict1[key], dict2[key], dict3[key]])
#             result[key] = merged_value
#             unique_branches += sub_branches
#         else:
#             values = []
#             if key in dict1 and dict1[key] is not None:
#                 if isinstance(dict1[key], list):
#                     values.extend(dict1[key])
#                 else:
#                     values.append(dict1[key])
#             if key in dict2 and dict2[key] is not None:
#                 if isinstance(dict2[key], list):
#                     values.extend(dict2[key])
#                 else:
#                     values.append(dict2[key])
#             if key in dict3 and dict3[key] is not None:
#                 if isinstance(dict3[key], list):
#                     values.extend(dict3[key])
#                 else:
#                     values.append(dict3[key])
#             result[key] = values
#             unique_branches += 1

#     return result, unique_branches
# dict1 = {
#     'A': {
#         'B': {
#             'C': [{'value': 1}]
#         },
#         'E': [{'value': 3}]
#     }
# }

# dict2 = {
#     'A': {
#         'B': {
#             'D': [{'value': 4}],
#             'F': [{'value': 5}]
#         },
#         'G': [{'value': 6}],
#         'Z': None
#     }
# }

# dict3 = {
#     'A': {
#         'B': {
#             'D': [{'value': 7}],
#             'H': [{'value': 8}]
#         },
#         'I': [{'value': 9}]
#     }
# }

# merged_dict, num_branches = merge_dicts([dict1, dict2, dict3])
# print(merged_dict)
# print("Number of unique branches:", num_branches)
# merged_dict, count_branches = merge_dicts(feature_sets)
# print(count_branches, merged_dict)
# with open(basedir + f'/feature_config.json', 'w') as file:
#         json.dump(kind_to_fc_parameters, file, indent=4)